In [23]:
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
import gensim 
from gensim.models import Word2Vec
import numpy as np

In [2]:
data = pd.read_csv('../phase1_movie_reviews-train.csv', dtype = str)

In [3]:
data['summary'] = data['summary'].astype(str).apply(text_to_word_sequence)
data['reviewText'] = data['reviewText'].astype(str).apply(text_to_word_sequence)

In [4]:
def cbow(data, num_features, min_word_count, num_workers, window): 
    model = gensim.models.Word2Vec(data, 
                                   min_count = min_word_count,
                                   size = num_features, 
                                   window = window, 
                                   workers = num_workers)
    return model

In [7]:
num_features = 100
min_word_count = 5
num_workers = 8
window = 5

model = cbow(data['reviewText'], num_features, min_word_count, num_workers, window)
cbow

<function __main__.cbow(data, num_features, min_word_count, num_workers, window)>

In [20]:
print("Total number of words in the vocabolary: ", model.wv.syn0.shape)

Total number of words in the vocabolary:  (54844, 100)


/Users/davidmortensen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [43]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,), dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index2word)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(reviews, model, num_features):
    """
    Calculate average feature vectors for all reviews
    """
    counter = 0.
    review_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')  # pre-initialize (for speed)
    
    for review in reviews:
        review_feature_vecs[counter] = make_feature_vec(review, model, num_features)
        counter = counter + 1.
    return review_feature_vecs

In [44]:
trainDataVecs = get_avg_feature_vecs(data['reviewText'], model, num_features)

/Users/davidmortensen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 4 columns):
polarity      90000 non-null object
summary       90000 non-null object
reviewText    90000 non-null object
year          90000 non-null object
dtypes: object(4)
memory usage: 2.7+ MB


In [45]:
x_vectors = model.wv

In [50]:
x_vectors.keys

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'keys'